In [163]:
from bs4 import BeautifulSoup
import pandas
import numpy as np
import pandas as pd
import csv
import requests
import re
import pprint
from datetime import datetime
from newspaper import Article
import nltk
from gtts import gTTS

#     """
#     beautiful soup parses through the user's HTML file and gets each 
#     link's article's body text
#     """


def get_articles_table(input_file):
    with open(input_file) as fp:
        soup = BeautifulSoup(fp, "lxml")

    all_articles_dictionary = {}
    pp = pprint.PrettyPrinter(indent=4)
    article_id = 0

    for link in soup.find_all("div", { "class" : "_2pin" }):
        article_id+=1
    #     pp.pprint(str(link.div.div.div.div.div))
        single_article_dictionary = {}
        post_section = 0

        for string in link.div.div.div.div.div:
            post_section+=1
            if post_section is 1:
                single_article_dictionary["hyperlink"] = str(string)[5:len(string)-7]
            if post_section is 2:
                single_article_dictionary["title"] = str(string)[5:len(string)-7]
            if post_section is 3:
                single_article_dictionary["publisher"] = str(string)[5:len(string)-7]
        all_articles_dictionary[article_id] = single_article_dictionary
    pd_articles = pd.DataFrame(all_articles_dictionary)
    return pd_articles

def get_articles_table_2(input_file):
    with open(input_file) as fp:
        soup = BeautifulSoup(fp, "lxml")

    all_articles_dictionary = {}
    article_id = 0
    
    
    for link in soup.find_all("div", { "class" : "uiBoxWhite" }):
        single_article_dictionary = {}
        post_section = 0
        link_content = link.find("div", { "class" : "_2pin" })
        if link_content == None:
            continue 
        for link_string in link_content.div.div.div.div.div:
            post_section += 1
            decoded_string = str(link_string).replace('\r\n', '\n')
            article_post = True

            if post_section is 1:
                if decoded_string[5:9] != "http":
                    post_section = 0
                    article_post = False
                    exit
                else:
                    single_article_dictionary["hyperlink"] = decoded_string[5:len(link_string)-7]
            if post_section is 2:
                single_article_dictionary["title"] = decoded_string[5:len(link_string)-7]
#                 if post_section is 3:
#                     single_article_dictionary["publisher"] = decoded_string[5:len(link_string)-7]
        # time published 
#         Oct 05, 2018 6:45pm
        
        single_article_dictionary["time_published"] = datetime.strptime(str(link.find("div", {"class":"_2lem"}).get_text()), "%b %d, %Y %I:%M%p")

        if "saved a link from" in str(link.find("div", {"class":"_2lel"}).get_text()):
            publisher_string = re.search('saved a (.*) post', str(link.find("div", {"class":"_2lel"}).get_text()))
            single_article_dictionary["publisher"] = publisher_string.group(1)[:len(publisher_string.group(1))-2].replace('link from','')
        else:
            single_article_dictionary["publisher"] = ""
            
        
        if article_post:
            article = Article(single_article_dictionary["hyperlink"])
            article.download()
            try:
                article.parse()
                single_article_dictionary["body"] = article.text
                tts = gTTS(single_article_dictionary["body"], lang='en')
                
                tts.save(single_article_dictionary["title"] + ".mp3")
                single_article_dictionary["audio_filename"] = single_article_dictionary["title"] + ".mp3"
            except:
                single_article_dictionary["body"] = "Article could not be found :("
                single_article_dictionary["audio_filename"] = "No audio :("
            article_id += 1               
            all_articles_dictionary[article_id] = single_article_dictionary
        if article_id >=100:
            break
    
    pd_articles = pd.DataFrame(all_articles_dictionary)
    return pd_articles
    #         print(link.div.div.div.div.div.div.get_text())
    #         print(link.div.div.div.div.div)
    #     dictionary[link.div.div.div.div.div.div.get_text()]
    # article_descriptions.head()
    


In [164]:
display(get_articles_table_2("your_saved_items.html"))




,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
audio_filename,Jordan Peterson doesn't understand postmoderni...,Noam Chomsky says US should stay in Syria to p...,"A Nobel Prize winner sold his medal for $765,0...",Starship Troopers: One of the Most Misundersto...,The Supreme Court: last line of defense for th...,Amazon's hourly workers lose monthly bonuses a...,"Corporate Profits Hit Record High, and 40% of ...","Childhood Guilt, Adult Depression?.mp3",Kavanaugh Has Become a Hero to the Incel Commu...,5 surprising things that North Korea and Irela...,...,No audio :(,No audio :(,No audio :(,No audio :(,No audio :(,No audio :(,Why Marx’s Capital Still Matters.mp3,Black farmers were intentionally sold fake see...,Fresh Proof That Strong Unions Help Reduce Inc...,"Enough Talk of Assassinating the President, Re..."
body,"Published on Apr 27, 2018\n\nPlease consider s...",ERBIL (Kurdistan 24) – The US should stay in n...,Leon Lederman sold his Nobel Prize medal in an...,TriStar Pictures; Touchstone Pictures\n\nWhen ...,This article was originally published as Chapt...,Amazon's minimum-wage increase for its hourly ...,72.7k SHARES Share Tweet\n\nThe latest after-t...,The researchers took brain scans of 145 school...,Brett Kavanaugh’s fate as a Justice on the Sup...,"\n\n\n\nTo begin, as a little disclaimer, nobo...",...,Article could not be found :(,Article could not be found :(,Article could not be found :(,Article could not be found :(,Article could not be found :(,Article could not be found :(,DH\n\nOne of the virtues of looking at capital...,"""The lawsuit against Stine Seed Company is wit...","Until now, the study of unions’ effect on ineq...","Enough Talk of Assassinating the President, Re..."
hyperlink,https://www.youtube.com/watch?v=cU1LhcEh8Ms,http://www.kurdistan24.net/en/news/13cf816e-8e...,https://www.thisisinsider.com/nobel-prize-winn...,http://www.theatlantic.com/entertainment/archi...,https://liberationschool.org/ch-6-the-supreme-...,https://www.cnbc.com/2018/10/03/amazon-hourly-...,https://gritpost.com/corporate-profits-america...,http://www.theatlantic.com/health/archive/2015...,https://www.thedailybeast.com/kavanaugh-has-be...,https://www.visitthedprk.org/5-surprising-thin...,...,http://thecharnelhouse.org/2013/07/14/a-soviet...,http://longreads.com/2018/07/12/petersons-comp...,https://jacobinmag.com/2018/01/winston-churchi...,http://fortune.com/2017/07/10/climate-change-g...,https://catalyst-journal.com/vol2/no1/between-...,http://intransigence.org/2018/07/09/fascism-de...,https://jacobinmag.com/2018/07/karl-marx-capit...,http://www.wmcactionnews5.com/story/38610463/b...,https://www.nytimes.com/2018/07/06/business/la...,https://thehardtimes.net/culture/enough-talk-o...
publisher,,Thimo Vijfschaft,Ben Norton,The Atlantic,Henry Ley,Capitalism Kills,Hampton Institute,The Atlantic,the,"President Xi, Delete This Account",...,the,Jordan Peterson's Neverneverland Ranch for Hi...,Jacobin Magazine,Chauncey Extreme,Liam Kelly,Ross Wolfe,Jacobin Magazine,Institute for Southern Studies,Laborers Local 79 - New York City,The Multicultural Center
time_published,2018-10-05 18:45:00,2018-10-05 17:00:00,2018-10-05 16:58:00,2018-10-05 12:17:00,2018-10-04 08:48:00,2018-10-03 22:22:00,2018-10-01 08:06:00,2018-09-30 17:40:00,2018-09-28 21:56:00,2018-09-28 10:46:00,...,2018-07-16 11:17:00,2018-07-14 22:47:00,2018-07-14 00:09:00,2018-07-13 13:26:00,2018-07-13 11:58:00,2018-07-12 14:55:00,2018-07-12 13:21:00,2018-07-12 11:45:00,2018-07-12 10:23:00,2018-07-12 10:22:00
title,Jordan Peterson doesn't understand postmodernism,Noam Chomsky says US should stay in Syria to p...,"A Nobel Prize winner sold his medal for $765,0...",Starship Troopers: One of the Most Misundersto...,The Supreme Court: last line of defense for th...,Amazon's hourly workers lose monthly bonuses a...,"Corporate Profits Hit Record High, and 40% of ...","Childhood Guilt, Adult Depression?",Kavanaugh Has Become a Hero to the Incel Commu...,5 surprising things that 

In [ ]:
get_articles_table("your_saved_items.html").to_csv("saved_items.csv")
get_articles_table_2("your_saved_items.html").to_csv("saved_items2.csv")

In [15]:
for link in soup.find_all("div", { "class" : "_3-96" }):
    article_id+=1
    
    if(link.div !=None):
        pp.pprint(str(link))
    single_article_dictionary = {}
    post_section = 0
    
#     if(link.div !=None):
        
#     for string in link.div.div.div.div.div.div:
#         post_section+=1
#         if post_section is 1:
#             single_article_dictionary["hyperlink"] = str(string)[5:len(string)-7]
#         if post_section is 2:
#             single_article_dictionary["title"] = str(string)[5:len(string)-7]
#         if post_section is 3:
#             single_article_dictionary["publisher"] = str(string)[5:len(string)-7]

#     all_articles_dictionary[article_id] = single_article_dictionary

# pd_articles = pd.DataFrame(all_articles_dictionary)
# pd_articles


NameError: name 'soup' is not defined

In [3]:
len("Joshua Jiang saved a link from")

30

In [115]:
import html2text
res = requests.get("http://www.kurdistan24.net/en/news/13cf816e-8e40-41c8-bb76-d453a3261d8b")
html = res.text
# extract the text from the html version
text = html2text.html2text(html)
print(text)

_menu_

[ ![](/images/facebook.png) ](https://www.facebook.com/K24English/) [
![](/images/twitter.png)](https://twitter.com/k24english) [
![](/images/instagram.png)](https://www.instagram.com/kurdistan24english/) [
![](/images/youtube.png)](https://www.youtube.com/channel/UCkWxVZRamwKWzhT_B0Ku9yA)

Live [TV](/en/LiveWatch/e9847195-3c42-47cb-9103-bf477c68bc8f)
[RADIO](/en/LiveRadio)

[![](/images/Logo.png)](/en/default) KURDISTAN24

  * Erbil
  *   * -

  * [Turkçe](/tr/SetCulture)
  * |
  * [English](/en/SetCulture)
  * |
  * [Kurdi](/ku/SetCulture)
  * |
  * [فارسی](/fa/SetCulture)
  * |
  * [عربي](/ar/SetCulture)
  * |
  * [كوردی](/so/SetCulture)

[ ![](/images/Logo.png) KURDISTAN 24 ](/so/)

  * [News ](/en/newslist)
    * [Kurdistan](/en/kurdistannews)
    * [Middle East](/en/middleeastnews)
    * [World](/en/worldnews)
  * [Analysis ](/en/Analysislist)
    * [Kurdistan](/en/Kurdistananalysis)
    * [Middle East](/en/middle_analysis)
    * [Features](/en/featuresanalysis)
  * [Econ

In [123]:
from newspaper import Article
import nltk

article = Article("http://www.kurdistan24.net/en/news/13cf816e-8e40-41c8-bb76-d453a3261d8b")
 
#1 . Download the article
article.download()
 
#2. Parse the article
article.parse()
 
#3. Fetch Author Name(s)
print(article.authors)
 
#4. Fetch Publication Date
print("Article Publication Date:")
print(article.publish_date)
#5. The URL of the Major Image
print("Major Image in the article:")
print(article.top_image)
 
#6. Natural Language Processing on Article to fetch Keywords
article.nlp()
print ("Keywords in the article")
print(article.keywords)
 
#7. Generate Summary of the article
print("Article Summary")
print(article.summary)    
    
#8. Generate Text of the article
print("Article Text")
print(article.text)

['Paul', 'Halmat Palani', 'Halima Galali', 'Mamta Singh']
Article Publication Date:
None
Major Image in the article:
http://kurdistan24.blob.core.windows.net/filemanager/resources/normals/2018/10/ChomskyGuardianGraemeRobertson-2.jpg
Keywords in the article
['turkey', 'kurdish', 'protect', 'syria', 'united', 'states', 'noam', 'deter', 'stay', 'chomsky', 'attacks', 'kurds', 'kurdistan']
Article Summary
ERBIL (Kurdistan 24) – The US should stay in northern Syria to deter attacks against Syrian Kurds, well-known American linguist and political activist Noam Chomsky said in an interview with the Intercept last week.
In my opinion, it makes sense for the United States to maintain a presence which would deter an attack on the Kurdish areas,” he said.
Chomsky noted that the Kurds have “succeeded in sustaining a functioning society with many decent elements” in Syria’s north.
“That does not change the fact that now the United States could, with a relatively small presence, deter attacks against

In [126]:
import pyttsx3
engine = pyttsx3.init()
engine.say("I will speak this text")
engine.runAndWait()

ModuleNotFoundError: No module named 'Foundation'